<a href="https://colab.research.google.com/github/Gautam12NI/102303739_UML_501_assignments/blob/main/102303739_Ass2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:

# Part I: Import Data
import pandas as pd

customers = pd.read_csv("AWCustomers.csv")
sales = pd.read_csv("AWSales.csv")

print("Customers shape:", customers.shape)
print("Sales shape:", sales.shape)
customers.head()

Customers shape: (18361, 24)
Sales shape: (18355, 3)


,CustomerID,Title,FirstName,MiddleName,LastName,Suffix,AddressLine1,AddressLine2,City,StateProvinceName,...,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,LastUpdated
0,21173,NaN,Chad,C,Yuan,NaN,7090 C. Mount Hood,NaN,Wollongong,New South Wales,...,Bachelors,Clerical,M,M,1,3,0,1,81916,2017-03-06
1,13249,NaN,Ryan,NaN,Perry,NaN,3651 Willow Lake Rd,NaN,Shawnee,British Columbia,...,Partial College,Clerical,M,M,1,2,1,2,81076,2017-03-06
2,29350,NaN,Julia,NaN,Thompson,NaN,1774 Tice Valley Blvd.,NaN,West Covina,California,...,Bachelors,Clerical,F,S,0,3,0,0,86387,2017-03-06
3,13503,NaN,Theodore,NaN,Gomez,NaN,2103 Baldwin Dr,NaN,Liverpool,England,...,Partial College,Skilled Manual,M,M,1,2,1,2,61481,2017-03-06
4,22803,NaN,Marshall,J,Shan,NaN,Am Gallberg 234,NaN,Werne,Nordrhein-Westfalen,...,Partial College,Skilled Manual,M,S,1,1,0,0,51804,2017-03-06


In [15]:

# Part I (a) & (b): Merge datasets and select features

merged = pd.merge(customers, sales, on="CustomerID")

# Create Age from BirthDate
merged["BirthDate"] = pd.to_datetime(merged["BirthDate"])
merged["Age"] = 2017 - merged["BirthDate"].dt.year

# Select relevant features (make independent copy)
selected = merged[[
    "Age","Education","Occupation","Gender","MaritalStatus",
    "HomeOwnerFlag","NumberCarsOwned","NumberChildrenAtHome",
    "TotalChildren","YearlyIncome","BikeBuyer"
]].copy()

selected.head()

,Age,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,BikeBuyer
0,30,Bachelors,Clerical,M,M,1,3,0,1,81916,1
1,45,Partial College,Clerical,M,M,1,2,1,2,81076,1
2,32,Bachelors,Clerical,F,S,0,3,0,0,86387,1
3,40,Partial College,Skilled Manual,M,M,1,2,1,2,61481,1
4,42,Partial College,Skilled Manual,M,S,1,1,0,0,51804,1


In [17]:

# Part II (a): Handling Null Values

num_cols = ["Age","YearlyIncome","NumberCarsOwned","NumberChildrenAtHome","TotalChildren"]
cat_cols = ["Education","Occupation","Gender","MaritalStatus"]

# Convert numeric columns to float
selected[num_cols] = selected[num_cols].astype(float)

print("Before:", selected.isnull().sum())

# Fill numeric with median
for col in num_cols:
    selected[col] = selected[col].fillna(selected[col].median())

# Fill categorical with mode
for col in cat_cols:
    selected[col] = selected[col].fillna(selected[col].mode()[0])

print("After:", selected.isnull().sum())

Before: Age                     0
Education               0
Occupation              0
Gender                  0
MaritalStatus           0
HomeOwnerFlag           0
NumberCarsOwned         0
NumberChildrenAtHome    0
TotalChildren           0
YearlyIncome            0
BikeBuyer               0
dtype: int64
After: Age                     0
Education               0
Occupation              0
Gender                  0
MaritalStatus           0
HomeOwnerFlag           0
NumberCarsOwned         0
NumberChildrenAtHome    0
TotalChildren           0
YearlyIncome            0
BikeBuyer               0
dtype: int64


In [18]:
# Part II (b): Normalization using MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
selected[num_cols] = scaler.fit_transform(selected[num_cols])
selected[num_cols].describe()


,Age,YearlyIncome,NumberCarsOwned,NumberChildrenAtHome,TotalChildren
count,18361.000000,18361.000000,18361.000000,18361.000000,18361.000000
mean,0.263249,0.416254,0.254060,0.112721,0.283463
std,0.160693,0.269933,0.182798,0.189652,0.309105
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.142857,0.245223,0.200000,0.000000,0.000000
50%,0.242857,0.320338,0.200000,0.000000,0.000000
75%,0.371429,0.545171,0.400000,0.333333,0.666667
max,1.000000,1.000000,1.000000,1.000000,1.000000


In [19]:

# Part II (c): Discretization (Binning)

# Keep safe copy before encoding
selected_before_encoding = selected.copy()

selected = selected.assign(
    TrueAge = 2017 - pd.to_datetime(merged["BirthDate"]).dt.year,
    AgeGroup = pd.cut(
        2017 - pd.to_datetime(merged["BirthDate"]).dt.year,
        bins=[0, 25, 45, 65, 100],
        labels=["Young","Adult","Middle-aged","Senior"]
    ),
    IncomeGroup = pd.cut(
        merged["YearlyIncome"],
        bins=[0, 30000, 70000, 150000],
        labels=["Low","Medium","High"]
    )
)

selected[["TrueAge","AgeGroup","YearlyIncome","IncomeGroup"]].head(10)


,TrueAge,AgeGroup,YearlyIncome,IncomeGroup
0,30,Adult,0.496842,High
1,45,Adult,0.489453,High
2,32,Adult,0.536172,High
3,40,Adult,0.317083,Medium
4,42,Adult,0.231958,Medium
5,42,Adult,0.321156,Medium
6,46,Middle-aged,0.083427,Medium
7,20,Young,0.320171,Medium
8,41,Adult,0.513714,High
9,41,Adult,0.012711,Low


In [20]:

# Part II (d): Standardization using StandardScaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
selected[num_cols] = scaler.fit_transform(selected[num_cols])
selected[num_cols].describe()

,Age,YearlyIncome,NumberCarsOwned,NumberChildrenAtHome,TotalChildren
count,1.836100e+04,1.836100e+04,1.836100e+04,1.836100e+04,1.836100e+04
mean,2.732112e-16,1.817861e-16,-3.130707e-16,-7.894489e-17,-5.959565e-17
std,1.000027e+00,1.000027e+00,1.000027e+00,1.000027e+00,1.000027e+00
min,-1.638260e+00,-1.542105e+00,-1.389881e+00,-5.943713e-01,-9.170694e-01
25%,-7.492263e-01,-6.336212e-01,-2.957460e-01,-5.943713e-01,-9.170694e-01
50%,-1.269031e-01,-3.553435e-01,-2.957460e-01,-5.943713e-01,-9.170694e-01
75%,6.732268e-01,4.775993e-01,7.983891e-01,1.163279e+00,1.239753e+00
max,4.584973e+00,2.162615e+00,4.080794e+00,4.678580e+00,2.318164e+00


In [21]:
# Part II (e): Binarization with One-Hot Encoding

selected = pd.get_dummies(
    selected,
    columns=["Education","Occupation","Gender","MaritalStatus","AgeGroup","IncomeGroup"],
    drop_first=True
)

selected.head()

,Age,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,BikeBuyer,TrueAge,Education_Graduate Degree,Education_High School,...,Occupation_Manual,Occupation_Professional,Occupation_Skilled Manual,Gender_M,MaritalStatus_S,AgeGroup_Adult,AgeGroup_Middle-aged,AgeGroup_Senior,IncomeGroup_Medium,IncomeGroup_High
0,-0.482516,1,1.892524,-0.594371,0.161342,0.298555,1,30,False,False,...,False,False,False,True,False,True,False,False,False,True
1,0.851033,1,0.798389,1.163279,1.239753,0.271180,1,45,False,False,...,False,False,False,True,False,True,False,False,False,True
2,-0.304710,0,1.892524,-0.594371,-0.917069,0.444261,1,32,False,False,...,False,False,False,False,True,True,False,False,False,True
3,0.406517,1,0.798389,1.163279,1.239753,-0.367401,1,40,False,False,...,False,False,True,True,False,True,False,False,True,False
4,0.584324,1,-0.295746,-0.594371,-0.917069,-0.682765,1,42,False,False,...,False,False,True,True,True,True,False,False,True,False


In [22]:

# Part III (a): Similarity between first 2 rows
import numpy as np

obj1 = selected.iloc[0]
obj2 = selected.iloc[1]

# Simple Matching Coefficient
smc = (obj1 == obj2).sum() / len(obj1)

# Jaccard Similarity
intersection = ((obj1 == 1) & (obj2 == 1)).sum()
union = ((obj1 == 1) | (obj2 == 1)).sum()
jaccard = intersection / union if union != 0 else 0

# Cosine Similarity
cosine = np.dot(obj1, obj2) / (np.linalg.norm(obj1) * np.linalg.norm(obj2))

print("Simple Matching Coefficient:", smc)
print("Jaccard Similarity:", jaccard)
print("Cosine Similarity:", cosine)


Simple Matching Coefficient: 0.6956521739130435
Jaccard Similarity: 0.8333333333333334
Cosine Similarity: 0.9965381198375434


In [23]:
# Part III (b): Correlation with Commute Distance (simulated since not in dataset)

np.random.seed(42)
merged["CommuteDistanceNum"] = np.random.choice([0.5,1.5,3.5,7.5,12], size=len(merged))

correlation = merged["CommuteDistanceNum"].corr(merged["YearlyIncome"])
print("Correlation between Commute Distance and Yearly Income:", correlation)

Correlation between Commute Distance and Yearly Income: 0.013948794730188998
